In [1]:
# upto 20200418


import pandas as pd
import numpy as np
import os
import datetime
import logging
import hashlib
import gc
import glob
logging.basicConfig(filename='BL_aggregate_18_months_sales_upto20200418_2020Q4_'+str(datetime.datetime.now().date())+'.log', level=logging.INFO)
logging.info('Started')

samplerows = None

lastdate_date = datetime.date(2020,4,18) # Recent Saturday
Beginning_12_months_ago = str(lastdate_date-datetime.timedelta(days=52*7-1)) # Sunday
Beginning_18_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*1.5-1)) # Sunday
Beginning_48_months_ago=str(lastdate_date-datetime.timedelta(days=52*7*4-1)) # Sunday

lastdate_str=str(lastdate_date)
print("Active Start on: "+Beginning_12_months_ago) #>=
print("Lapsed & Store_allocation Start on: "+Beginning_18_months_ago) #>=
print("Lapsed 19-48 Start on: "+Beginning_48_months_ago) #>=

def recursive_file_gen(root_folder):
    for root, dirs, files in os.walk(root_folder):
        for file in files:
            yield os.path.join(root, file)
            
folder_write = '/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q2/output_'+str(datetime.datetime.now().date())+'/'
try:
    os.stat(folder_write)
except:
    os.mkdir(folder_write)
    
os.getcwd()

Active Start on: 2019-04-21
Lapsed & Store_allocation Start on: 2018-10-21
Lapsed 19-48 Start on: 2016-04-24


'/home/jian/Projects/Big_Lots/Live_Ramp/Quarterly_Update_2020Q2'

In [2]:
historical_item_files=glob.glob("/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/*.txt")
historical_item_files=[x for x in historical_item_files if x.split("ailySalesHistory")[1][:8]>=Beginning_18_months_ago.replace("-","")]
daily_item_files_2019_20202=list(recursive_file_gen("/home/jian/BigLots/2019_by_weeks/"))
daily_item_files_2019_20202_part2=list(recursive_file_gen("/home/jian/BigLots/2020_by_weeks/"))
daily_item_files_2019_20202=daily_item_files_2019_20202+daily_item_files_2019_20202_part2
del daily_item_files_2019_20202_part2

daily_item_files_2019_20202=[x for x in daily_item_files_2019_20202 if x[-4:]==".txt" and "daily" in x.lower()]


historical_item_files.sort()
daily_item_files_2019_20202.sort()


In [3]:
# Piece 1: item 2018-10-21 to 2019-02-09

df=pd.read_table(historical_item_files[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_historical_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
# min_date_historical_item: 2018-08-05


df=pd.read_table(historical_item_files[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_historical_item: "+str(df['transaction_dt'].max()))
del df
gc.collect()
# max_date_historical_item: 2019-02-09

min_date_historical_item: 2018-10-21
max_date_historical_item: 2019-02-09


0

In [4]:
# Piece 2: item 2019-02-10 to 2020-01-11

daily_item_files_2019_20202=[x for x in daily_item_files_2019_20202 if x.split("_weeks/MediaStorm_")[1][:10]>="2019-02-10"]


#QC
df=pd.read_table(daily_item_files_2019_20202[0],dtype=str,sep="|",usecols=['transaction_dt'])
print("min_date_daily_item: "+str(df['transaction_dt'].min()))
del df
gc.collect()
# min_date_daily_item: 2019-02-10


df=pd.read_table(daily_item_files_2019_20202[-1],dtype=str,sep="|",usecols=['transaction_dt'])
print("max_date_daily_item: "+str(df['transaction_dt'].max()))
del df
gc.collect()
# max_date_daily_item: 2020-01-11

min_date_daily_item: 2019-02-10
max_date_daily_item: 2020-04-18


0

In [5]:
list_all_files=historical_item_files+daily_item_files_2019_20202

print(len(list_all_files))

78


In [6]:
list_all_files

['/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181208.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181215.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181222.txt',
 '/home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHi

In [7]:
list_df_transactions_18_months=[]
i_counter=0
for file in list_all_files:
    try:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','item_transaction_amt'])
        df=df.rename(columns={"item_transaction_amt":"sales"})
    except:
        df=pd.read_table(file,dtype=str,sep="|",usecols=['customer_id_hashed','transaction_dt','transaction_id','location_id','subclass_transaction_amt'])
        df=df.rename(columns={"subclass_transaction_amt":"sales"})
    df=df[pd.notnull(df['customer_id_hashed'])]
    df['sales']=df['sales'].astype(float)
    df_sales=df.groupby(['customer_id_hashed','transaction_dt'])['sales'].sum().to_frame().reset_index()
    
    df_trans=df[['customer_id_hashed','transaction_dt','transaction_id','location_id']].drop_duplicates()
    df_trans['trans']=1
    df_trans=df_trans.groupby(['customer_id_hashed','transaction_dt'])['trans'].sum().to_frame().reset_index()
    df=pd.merge(df_sales,df_trans,on=["customer_id_hashed",'transaction_dt'],how="outer")

    print(i_counter,df['transaction_dt'].min(),df['transaction_dt'].max(),datetime.datetime.now(),file)
    i_counter+=1
    list_df_transactions_18_months.append(df)

0 2018-10-21 2018-10-27 2020-04-22 15:21:35.954132 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181027.txt
1 2018-10-28 2018-11-03 2020-04-22 15:22:37.165745 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181103.txt
2 2018-11-04 2018-11-10 2020-04-22 15:23:40.903168 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181110.txt
3 2018-11-11 2018-11-17 2020-04-22 15:25:10.731065 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181117.txt
4 2018-11-18 2018-11-24 2020-04-22 15:26:58.661412 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181124.txt
5 2018-11-25 2018-12-01 2020-04-22 15:28:38.786506 /home/jian/BigLots/hist_daily_data_itemlevel_decompressed/MediaStormDailySalesHistory20181201.txt
6 2018-12-02 2018-12-08 2020-04-22 15:30:27.611469 /home/jian/BigLots/hist_daily_data_itemlevel_decompress

55 2019-11-10 2019-11-16 2020-04-22 16:26:57.124958 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-16/MediaStormDailySales20191119-112232-478.txt
56 2019-11-17 2019-11-23 2020-04-22 16:28:22.346262 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-23/MediaStormDailySales20191126-112901-552.txt
57 2019-11-24 2019-11-30 2020-04-22 16:30:19.981851 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-11-30/MediaStormDailySales20191203.txt
58 2019-12-01 2019-12-07 2020-04-22 16:32:15.865426 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-07/MediaStormDailySales20191211.txt
59 2019-12-08 2019-12-14 2020-04-22 16:34:15.031627 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-14/MediaStormDailySales20191217-195625-000.txt
60 2019-12-15 2019-12-21 2020-04-22 16:36:21.624085 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-21/MediaStormDailySales20191226-122746-000.txt
61 2019-12-22 2019-12-28 2020-04-22 16:38:03.253602 /home/jian/BigLots/2019_by_weeks/MediaStorm_2019-12-28/MediaStor

In [16]:
print("len(list_df_transactions_18_months):" +str(len(list_df_transactions_18_months)))
df_transactions_18_months=pd.concat(list_df_transactions_18_months[:])
del list_df_transactions_18_months

gc.collect()
print("df_transactions_18_months.shape:" +str(df_transactions_18_months.shape))
print("df_transactions_18_months['transaction_dt'].nunique():" +str(df_transactions_18_months['transaction_dt'].nunique()))
print("df_transactions_18_months['customer_id_hashed'].nunique():" +str(df_transactions_18_months['customer_id_hashed'].nunique()))
print("date_range: "+str(df_transactions_18_months['transaction_dt'].min())+" | "+str(df_transactions_18_months['transaction_dt'].max()))

df_transactions_18_months.shape:(113785770, 4)
df_transactions_18_months['transaction_dt'].nunique():546
df_transactions_18_months['customer_id_hashed'].nunique():22812016
date_range: 2018-10-21 | 2020-04-18


In [17]:
df_transactions_18_months.to_csv(folder_write+"BL_sales_trans_by_id_date_JL_"+str(datetime.datetime.now().date())+".csv",index=False)

In [18]:
df_transactions_18_months.head(2)

,customer_id_hashed,transaction_dt,sales,trans
0,000001dadc0265bf9d250566d74e0006323f18b5826641...,2018-10-27,42.35,1
1,00000950dea4a869e9fe70d823444d418c5abebbd8e830...,2018-10-23,22.20,1


In [19]:
df_transactions_18_months['sales'].sum()

5192910791.289989

In [20]:
365*1.5

547.5

In [21]:
df_transactions_18_months.shape,df_transactions_18_months['customer_id_hashed'].nunique()

((113785770, 4), 22812016)

In [22]:
113785770/22812016

4.987975196931301